## COMP30830 - Software Engineering
### Project: DublinBikes  
Group work by:  

**Xi Jiang  
Yuqian Shu  
Yi Zhang**

### Machine Learning prediction model
In this notebook we develop a ML model to predict two key values that are functional to our web application:  
  
**number of available bikes for a selected station  
number of available bike stands for a selected station**  
  
The model is trained on data fetched from an Amazon RDS database, where we store historical data referring to:
  
real-time information that Dublin Bikes makes available through their web API  
Dublin weather information from OpenWeather, obtained by a web API  
***N.B.:*** all database data are kept up-to-date.

### Getting started
First of all, we need to import Python packages that are required to our model, as well as fetching the data from the Amazon RDS database.

#### Import required packages
A number of Python packages are required for our model in order to work, so we import them in our Notebook.

In [1]:
import pickle 
#import seaborn as sns
#import matplotlib.pyplot as plt
import pandas as pd 
#import numpy as np 
#import json
import pymysql

#from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import PolynomialFeatures
#from sklearn.pipeline import make_pipeline
#from patsy import dmatrices
from sklearn import metrics
#from sklearn.model_selection import cross_validate

#### Connect to Amazon RDS database
After having stored all the necessary credentials in specific variables, we connect to the database providing error-handling in case of connection issues.

In [2]:
# Attempt connection to database
# Print a statement on the screen to check whether the connection is working
try:
    con = pymysql.connect(host='dublinbike.cczltqdfsp1t.eu-west-1.rds.amazonaws.com', user='root', passwd='shuyuqian',db='dublin')
    print('+=========================+')
    print('|    TRYING TO CONNECT    |')
    print('+=========================+')
    print('|        SUCCESS!         |')
    print('+=========================+')
    
# Exit if connection not working   
except Exception as e:
        sys.exit(e)

+=========================+
|    TRYING TO CONNECT    |
+=========================+
|        SUCCESS!         |
+=========================+


#### Data fetching
Bikes information
We store real-time data from Dublin Bikes in a relation named "availability" in the RDS database schema.  
We store weather data from OpenWeather in a relation named "weather" in the RDS database schema.
  
We fetch Pandas dataframe object and we examine it.

***N.B.:***We started scrapping the bikes data on 02/03/2020

In [3]:
# Create dataframe and store data running SQL query
df_AllStations = pd.read_sql_query("select *,cast(str_to_date(availability.datetime,'%d-%b-%Y (%H:%i:%s.%f)' )as datetime) as datetimeB FROM dublin.availability, dublin.Weather having abs(datetimeB-Weather.dateTime)<10800 and Weather.dateTime<'2020/4/07'", con)

# Examine dataframe object, show first 10 rows
df_AllStations.head(5)

,datetime,number,available_bikes,available_bike_stands,last_update,dateTime,weatherID,weatherMain,weatherDescr,weatherIcon,temperature,pressure,humidity,tempMin,tempMax,visibility,windSpeed,windDeg,clouds,datetimeB
0,03-Mar-2020 (10:29:28.225236),42,0,30,0,2020-03-03 10:33:00,801,Clouds,few clouds,02d,6,999,80,5,7,10000,10,270,20,2020-03-03 10:29:28
1,03-Mar-2020 (10:29:28.225236),30,0,20,0,2020-03-03 10:33:00,801,Clouds,few clouds,02d,6,999,80,5,7,10000,10,270,20,2020-03-03 10:29:28
2,03-Mar-2020 (10:29:28.225236),54,11,22,0,2020-03-03 10:33:00,801,Clouds,few clouds,02d,6,999,80,5,7,10000,10,270,20,2020-03-03 10:29:28
3,03-Mar-2020 (10:29:28.225236),108,0,40,0,2020-03-03 10:33:00,801,Clouds,few clouds,02d,6,999,80,5,7,10000,10,270,20,2020-03-03 10:29:28
4,03-Mar-2020 (10:29:28.225236),56,31,9,0,2020-03-03 10:33:00,801,Clouds,few clouds,02d,6,999,80,5,7,10000,10,270,20,2020-03-03 10:29:28


In [4]:
df_AllStations.shape

(166210, 20)

In [5]:
#create column for day of the week
df_AllStations['weekday'] = df_AllStations['datetimeB'].dt.dayofweek
df_AllStations['hour'] = df_AllStations['datetimeB'].dt.hour
df_AllStations['minutes'] = df_AllStations['datetimeB'].dt.minute

In [6]:
#use number to replace weekdays
df_AllStations['weekday'] = df_AllStations['weekday'].replace(0, 'Monday')
df_AllStations['weekday'] = df_AllStations['weekday'].replace(1, 'Tuesday')
df_AllStations['weekday'] = df_AllStations['weekday'].replace(2, 'Wednesday')
df_AllStations['weekday'] = df_AllStations['weekday'].replace(3, 'Thursday')
df_AllStations['weekday'] = df_AllStations['weekday'].replace(4, 'Friday')
df_AllStations['weekday'] = df_AllStations['weekday'].replace(5, 'Saturday')
df_AllStations['weekday'] = df_AllStations['weekday'].replace(6, 'Sunday')

In [7]:
#show all the columns of the dataframe
df_AllStations.columns

Index(['datetime', 'number', 'available_bikes', 'available_bike_stands',
       'last_update', 'dateTime', 'weatherID', 'weatherMain', 'weatherDescr',
       'weatherIcon', 'temperature', 'pressure', 'humidity', 'tempMin',
       'tempMax', 'visibility', 'windSpeed', 'windDeg', 'clouds', 'datetimeB',
       'weekday', 'hour', 'minutes'],
      dtype='object')

### Dummy coding(categorical -> continuous)

We need to incorporate the following information (variables) in our model, but they are not immediately usable as they come in a categorical form:  
**cloud coverage  
day of the week**  
Thus, we transform them in a series of dummy variables. The process is known as "dummy coding".

In [8]:
# Create a separate dataframe with days of the week (categorical)
data_input1 = pd.DataFrame(df_AllStations['weekday'])

# Create a separate dataframe with cloud coverage information (categorical)
data_input2 = pd.DataFrame(df_AllStations['weatherMain'])

# Concatenate the two dataframes in the main one
dummy = pd.get_dummies(data_input1)
dummy_2 = pd.get_dummies(data_input2)
df_AllStations = pd.concat([df_AllStations,dummy],axis=1)
df_AllStations = pd.concat([df_AllStations,dummy_2],axis=1)

In [9]:
## Examine dataframe object, show first rows
df_AllStations.head()

,datetime,number,available_bikes,available_bike_stands,last_update,dateTime,weatherID,weatherMain,weatherDescr,weatherIcon,...,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday,weatherMain_Clear,weatherMain_Clouds,weatherMain_Drizzle,weatherMain_Mist,weatherMain_Rain
0,03-Mar-2020 (10:29:28.225236),42,0,30,0,2020-03-03 10:33:00,801,Clouds,few clouds,02d,...,0,0,0,1,0,0,1,0,0,0
1,03-Mar-2020 (10:29:28.225236),30,0,20,0,2020-03-03 10:33:00,801,Clouds,few clouds,02d,...,0,0,0,1,0,0,1,0,0,0
2,03-Mar-2020 (10:29:28.225236),54,11,22,0,2020-03-03 10:33:00,801,Clouds,few clouds,02d,...,0,0,0,1,0,0,1,0,0,0
3,03-Mar-2020 (10:29:28.225236),108,0,40,0,2020-03-03 10:33:00,801,Clouds,few clouds,02d,...,0,0,0,1,0,0,1,0,0,0
4,03-Mar-2020 (10:29:28.225236),56,31,9,0,2020-03-03 10:33:00,801,Clouds,few clouds,02d,...,0,0,0,1,0,0,1,0,0,0


### Check dataframe state

#### New columns
The new dummy coded variables have been concatenated at the end of the dataframe, so we now check our dataframe shape.

In [10]:
# Show number of rows and columns of the dataframe
print("The dataset has %s rows and %s columns." % df_AllStations.shape)

The dataset has 166210 rows and 35 columns.


#### Data types
As a further check, we analyze the data type in our dataframe.

In [11]:
df_AllStations.dtypes

datetime                         object
number                            int64
available_bikes                   int64
available_bike_stands             int64
last_update                       int64
dateTime                 datetime64[ns]
weatherID                         int64
weatherMain                      object
weatherDescr                     object
weatherIcon                      object
temperature                       int64
pressure                          int64
humidity                          int64
tempMin                           int64
tempMax                           int64
visibility                        int64
windSpeed                         int64
windDeg                           int64
clouds                            int64
datetimeB                datetime64[ns]
weekday                          object
hour                              int64
minutes                           int64
weekday_Friday                    uint8
weekday_Monday                    uint8


### ML model
In order to predict our two target variables  

**number of available bikes  
number of available bike stands**  
we need two differnt ML models that are trained in the following section.  
After having tested the regression model as a viable alternative, we decided to implement a *Random Forest* classifier model, as it proves to be a more effective predictor.
  
 ***N.B.:*** we train our models on a random selection of 2/3 of the original dataset. We perform testing on the remain

#### (1)Predict the number of available bikes

**model training**

In [12]:
# Select model features and store them in a new dataframe
input_model = pd.DataFrame(df_AllStations[['number','temperature','windSpeed','hour','weekday_Friday', 'weekday_Monday',
       'weekday_Saturday', 'weekday_Sunday', 'weekday_Thursday',
       'weekday_Tuesday', 'weekday_Wednesday']])
input_model = pd.concat([input_model,dummy_2],axis=1)

# Define target variable
output = df_AllStations['available_bikes']

In [13]:
# Split dataset to train and test
X_train,X_test,Y_train,Y_test=train_test_split(input_model,output,test_size=0.33,random_state=42)
print("Training the model on %s rows and %s columns." % X_train.shape)

Training the model on 111360 rows and 16 columns.


In [14]:
# Instantiate RandomForestRegressor object calling 10 decision tree models
rf = RandomForestRegressor(n_estimators=10)

# Train the model
rf.fit(X_train, Y_train)

print("Testing the model on %s rows." % Y_test.shape[0])

Testing the model on 54850 rows.


**model testing**  
Using the trained model to predict the target feature availablebikes on the testing dataset

In [15]:
# Get prediction for test cases
prediction = rf.predict(X_test)

#make a new datafram to show the predicted available bikes
DF_Predicated = pd.DataFrame(prediction, columns=['Predicted'])

#convert all the data for testing to a new datafram
DF_Alltest = df_AllStations.iloc[Y_test]

#reset the index
DF_Bikes = pd.DataFrame(DF_Alltest['available_bikes']).reset_index(drop=True)

#to get a clear comparisaon, concatenate two new datafram
actual_vs_predicted= pd.concat([DF_Bikes,DF_Predicated], axis=1)

In [16]:
print("\nPredictions with multiple linear regression: \n")
actual_vs_predicted


Predictions with multiple linear regression: 



,available_bikes,Predicted
0,21,14.500000
1,9,9.066667
2,17,12.316667
3,0,2.780000
4,29,10.075000
...,...,...
54845,0,0.000000
54846,0,10.323333
54847,0,2.100000
54848,11,0.650000


**Model evaluation**  
In order to evaluate the prediction effectiveness of our model, we compute the mean-absolute error, the mean squared error,the root-mean-square deviation and R2.

In [17]:
def printMetrics(testActualVal, predictions):
    #classification evaluation measures
    print('Error Evaluation')
    print('==============================================================================')
    print("MAE (Mean Absolute Error): ", metrics.mean_absolute_error(testActualVal, predictions))
    print("MSE (Mean Squared Error): ", metrics.mean_squared_error(testActualVal, predictions))
    print("RMSE (Root Mean Squared Error): ", metrics.mean_squared_error(testActualVal, predictions)**0.5)
    print("R2: ", metrics.r2_score(testActualVal, predictions))

In [18]:
printMetrics(Y_test, prediction)

Error Evaluation
MAE (Mean Absolute Error):  1.3868445433553593
MSE (Mean Squared Error):  6.9584504010803245
RMSE (Root Mean Squared Error):  2.6378874883285537
R2:  0.9133136854343111


The R² value implies that there is 94% less variation around the line than the mean. In other words, the relationship between the input variables and the number of available bikes accounts for 94% of the variation.

**Module integration**  
In order to connect the ML model to our Flask web application, we need to produce a 'prediction-data' file from the trained model using the ***Pickle*** Python module.  
Pickle allows us to store the prediction model in a file that we save on the server, in order to be used by the application to actually deliver a prediction based on the requested stations by the user.

In [19]:
pickle.dump(rf,open('final_prediction_bike.pickle', 'wb'))


# This is not strictly functional to the application
random_forest = pickle.load(open("final_prediction_bike.pickle", "rb"))

#### (2)Predict the number of available bikes

**model training**

In [20]:
# Select model features and store them in a new dataframe
input_model = pd.DataFrame(df_AllStations[['number','temperature','windSpeed','hour','weekday_Friday', 'weekday_Monday',
       'weekday_Saturday', 'weekday_Sunday', 'weekday_Thursday',
       'weekday_Tuesday', 'weekday_Wednesday']])
input_model = pd.concat([input_model,dummy_2],axis=1)

# Define target variable
output = df_AllStations['available_bike_stands']

In [21]:
# Split dataset to train and test
X_train,X_test,Y_train,Y_test=train_test_split(input_model,output,test_size=0.33,random_state=42)
print("Training the model on %s rows and %s columns." % X_train.shape)

# Instantiate RandomForestRegressor object calling 10 decision tree models
rf2 = RandomForestRegressor(n_estimators=10)

# Train the model
rf2.fit(X_train, Y_train)

print("Testing the model on %s rows." % Y_test.shape[0])

Training the model on 111360 rows and 16 columns.
Testing the model on 54850 rows.


**model testing**
  
 Using the trained model to predict the target feature availablebikes on the testing dataset

In [22]:
# Get prediction for test cases
prediction = rf2.predict(X_test)

#make a new datafram to show the predicted available bikes
DF_Predicated = pd.DataFrame(prediction, columns=['Predicted'])

#convert all the data for testing to a new datafram
DF_Alltest = df_AllStations.iloc[Y_test]

#reset the index
DF_Stands = pd.DataFrame(DF_Alltest['available_bike_stands']).reset_index(drop=True)

#to get a clear comparisaon, concatenate two new datafram
actual_vs_predicted= pd.concat([DF_Stands,DF_Predicated], axis=1)

In [23]:
print("\nPredictions with multiple linear regression: \n")
actual_vs_predicted


Predictions with multiple linear regression: 



,available_bike_stands,Predicted
0,15,25.358333
1,13,24.133333
2,14,7.366667
3,30,27.580000
4,22,30.000000
...,...,...
54845,26,40.000000
54846,30,8.783333
54847,21,39.000000
54848,15,23.977381


**model evaluation**  
In order to evaluate the prediction effectiveness of our model, we compute the mean-absolute error, the mean squared error,the root-mean-square deviation and R2.

In [24]:
def printMetrics(testActualVal, predictions):
    #classification evaluation measures
    print('Error Evaluation of stands')
    print('==============================================================================')
    print("MAE (Mean Absolute Error): ", metrics.mean_absolute_error(testActualVal, predictions))
    print("MSE (Mean Squared Error): ", metrics.mean_squared_error(testActualVal, predictions))
    print("RMSE (Root Mean Squared Error): ", metrics.mean_squared_error(testActualVal, predictions)**0.5)
    print("R2: ", metrics.r2_score(testActualVal, predictions))

In [25]:
printMetrics(Y_test, prediction)

Error Evaluation of stands
MAE (Mean Absolute Error):  1.3916201635830838
MSE (Mean Squared Error):  7.018051110943763
RMSE (Root Mean Squared Error):  2.649160453982311
R2:  0.9363235132700268


The R² value implies that there is 94% less variation around the line than the mean. In other words, the relationship between the input variables and the number of available bikes accounts for 94% of the variation.

**Module integration**  
In order to connect the ML model to our Flask web application, we need to produce a 'prediction-data' file from the trained model using the ***Pickle*** Python module.  
Pickle allows us to store the prediction model in a file that we save on the server, in order to be used by the application to actually deliver a prediction based on the requested stations by the user.

In [26]:
pickle.dump(rf2,open('final_prediction_bike_stands.pickle', 'wb'))

# This is not strictly functional to the application
random_forest_stands=pickle.load(open("final_prediction_bike_stands.pickle", "rb"))